In [369]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.pandas.set_option('max_columns',None)

In [370]:
movie_df = pd.read_csv('movies.csv')
rating_df = pd.read_csv('ratings.csv')

In [371]:
movie_df.head()
movie_df.drop(columns=['genres'],inplace=True)

In [372]:
movie_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [373]:
movie_df.rename(columns={'movieId':'userId'},inplace=True)

In [374]:
movie_df.columns

Index(['userId', 'title'], dtype='object')

In [375]:
rating_df.drop(columns=['timestamp'],inplace=True)

In [376]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


**Combining the both dataframe on the same columns**

In [377]:
data = pd.merge(rating_df,movie_df,on='userId')

In [378]:
data.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Toy Story (1995)
2,1,6,4.0,Toy Story (1995)
3,1,47,5.0,Toy Story (1995)
4,1,50,5.0,Toy Story (1995)


In [379]:
round(data.describe())

,userId,movieId,rating
count,88872.0,88872.0,88872.0
mean,323.0,20386.0,3.0
std,180.0,36319.0,1.0
min,1.0,1.0,0.0
25%,177.0,1214.0,3.0
50%,325.0,3147.0,4.0
75%,474.0,8711.0,4.0
max,610.0,193609.0,5.0


In [380]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88872 entries, 0 to 88871
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   88872 non-null  int64  
 1   movieId  88872 non-null  int64  
 2   rating   88872 non-null  float64
 3   title    88872 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 3.4+ MB


In [381]:
data.columns

Index(['userId', 'movieId', 'rating', 'title'], dtype='object')

In [382]:
data.isnull().sum()

userId     0
movieId    0
rating     0
title      0
dtype: int64

In [383]:
rating_count=data.groupby('title').count()['rating']
rating_count = rating_count.sort_values(ascending=False)
rating_count.head()

title
Air Up There, The (1994)      2698
Wild Bunch, The (1969)        2478
In the Line of Fire (1993)    2108
Fearless (1993)               1864
Man of the House (1995)       1346
Name: rating, dtype: int64

In [384]:
rating_data_count = pd.DataFrame(rating_count)

In [385]:
rating_data_count.reset_index(inplace=True)

In [386]:
rating_data_count.columns =['title','Count of Rating']

In [387]:
rating_data_count.head()

,title,Count of Rating
0,"Air Up There, The (1994)",2698
1,"Wild Bunch, The (1969)",2478
2,In the Line of Fire (1993),2108
3,Fearless (1993),1864
4,Man of the House (1995),1346


**Now mearge with original dataframe**

In [388]:
data = data.merge(rating_count,left_on='title',right_on='title',how='left')
#In this mearge we have taken the portion left_on and right on and how is left side'

In [389]:
data.columns = ['userId','movieId','rating','title','rating_count']

In [390]:
data.head()

,userId,movieId,rating,title,rating_count
0,1,1,4.0,Toy Story (1995),232
1,1,3,4.0,Toy Story (1995),232
2,1,6,4.0,Toy Story (1995),232
3,1,47,5.0,Toy Story (1995),232
4,1,50,5.0,Toy Story (1995),232


In [391]:
data['rating_count'].describe()

count    88872.000000
mean       630.979454
std        681.362937
min         20.000000
25%        152.000000
50%        386.000000
75%        862.000000
max       2698.000000
Name: rating_count, dtype: float64

In [392]:
popularity_threshold = 50
data = data.query('rating_count >= @popularity_threshold')
data.head()

,userId,movieId,rating,title,rating_count
0,1,1,4.0,Toy Story (1995),232
1,1,3,4.0,Toy Story (1995),232
2,1,6,4.0,Toy Story (1995),232
3,1,47,5.0,Toy Story (1995),232
4,1,50,5.0,Toy Story (1995),232


In [393]:
data.shape

(82924, 5)

### Lets create the Pivot Matrix.

In [394]:
pivot_matrix = data.pivot_table(values='rating',columns='userId',index='title').fillna(0)

In [395]:
pivot_matrix.head()

userId,1,4,6,7,10,11,15,16,17,18,19,20,21,22,23,24,27,28,29,31,32,34,36,38,39,40,41,42,43,45,47,50,52,57,58,62,63,64,66,68,70,73,74,75,76,78,79,80,82,83,86,88,89,93,94,95,96,99,100,101,102,103,104,105,108,110,111,112,113,116,117,119,121,122,123,125,129,132,135,137,140,141,144,149,151,152,153,156,159,160,165,166,168,169,170,171,174,177,178,179,181,183,184,186,187,190,191,195,198,199,201,202,204,210,211,212,213,215,216,217,219,220,222,223,224,225,227,229,230,232,233,234,235,237,239,240,241,246,247,248,249,254,256,260,261,262,263,265,266,267,273,274,275,279,280,282,284,287,288,290,292,294,298,301,303,304,305,306,307,308,310,312,313,314,316,317,318,321,322,325,326,328,330,331,332,334,336,337,339,341,342,343,344,345,346,348,351,352,353,354,356,357,359,361,362,365,367,368,369,370,372,373,376,377,380,381,382,384,385,386,387,390,391,393,405,408,409,410,412,413,414,415,416,417,418,419,420,422,424,425,426,427,428,429,430,432,434,436,437,438,445,446,448,450,452,453,455,457,458,460,464,466,469,470,474,475,476,477,479,480,482,484,486,487,488,489,490,491,492,493,495,497,500,504,509,510,511,512,514,517,520,522,523,524,526,527,528,532,534,541,542,543,546,551,552,553,555,558,562,563,564,567,573,577,579,580,583,585,586,587,588,590,592,593,594,596,597,599,600,602,605,606,608,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8 Seconds (1994),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.092199,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Above the Rim (1994),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.84127,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

### Let begine with the KNN Modeling.

- https://machinelearningmastery.com/sparse-matrices-for-machine-learning/

In [396]:
from scipy.sparse import csr_matrix

In [397]:
movie_recomdation_df_matrix = csr_matrix(pivot_matrix.values) #making the array and comress the row matrix

**It is very much good in hadeling the Sparse Matrix (max Null value array) so that time and computational ppower will not get suffer** 

In [398]:
from sklearn.neighbors import NearestNeighbors

In [399]:
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(movie_recomdation_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [400]:
movie_recomdation_df_matrix.shape

(334, 334)

### Model got fitted with cosine similarity where p = 2 .It means Euclediean Distance.

In [401]:
index = np.random.choice(pivot_matrix.shape[0])
print(index)

#Random sampling has been done on this basis we will predict the nearest neibour with the help of Eucleadean Distances.

53


In [402]:
distances, indices = model_knn.kneighbors(pivot_matrix.iloc[index,:].values.reshape(1,-1), n_neighbors = 13)

In [403]:
distances.flatten()

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [404]:
pivot_matrix.head()

userId,1,4,6,7,10,11,15,16,17,18,19,20,21,22,23,24,27,28,29,31,32,34,36,38,39,40,41,42,43,45,47,50,52,57,58,62,63,64,66,68,70,73,74,75,76,78,79,80,82,83,86,88,89,93,94,95,96,99,100,101,102,103,104,105,108,110,111,112,113,116,117,119,121,122,123,125,129,132,135,137,140,141,144,149,151,152,153,156,159,160,165,166,168,169,170,171,174,177,178,179,181,183,184,186,187,190,191,195,198,199,201,202,204,210,211,212,213,215,216,217,219,220,222,223,224,225,227,229,230,232,233,234,235,237,239,240,241,246,247,248,249,254,256,260,261,262,263,265,266,267,273,274,275,279,280,282,284,287,288,290,292,294,298,301,303,304,305,306,307,308,310,312,313,314,316,317,318,321,322,325,326,328,330,331,332,334,336,337,339,341,342,343,344,345,346,348,351,352,353,354,356,357,359,361,362,365,367,368,369,370,372,373,376,377,380,381,382,384,385,386,387,390,391,393,405,408,409,410,412,413,414,415,416,417,418,419,420,422,424,425,426,427,428,429,430,432,434,436,437,438,445,446,448,450,452,453,455,457,458,460,464,466,469,470,474,475,476,477,479,480,482,484,486,487,488,489,490,491,492,493,495,497,500,504,509,510,511,512,514,517,520,522,523,524,526,527,528,532,534,541,542,543,546,551,552,553,555,558,562,563,564,567,573,577,579,580,583,585,586,587,588,590,592,593,594,596,597,599,600,602,605,606,608,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8 Seconds (1994),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.092199,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Above the Rim (1994),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.84127,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [405]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(pivot_matrix.index[index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, pivot_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Brothers McMullen, The (1995):

1: Nick of Time (1995), with distance of 1.0:
2: One Fine Day (1996), with distance of 1.0:
3: Once Were Warriors (1994), with distance of 1.0:
4: Once Upon a Time... When We Were Colored (1995), with distance of 1.0:
5: Now and Then (1995), with distance of 1.0:
6: No Escape (1994), with distance of 1.0:
7: Nine Months (1995), with distance of 1.0:
8: Nina Takes a Lover (1994), with distance of 1.0:
9: Pagemaster, The (1994), with distance of 1.0:
10: Nightmare Before Christmas, The (1993), with distance of 1.0:
11: National Lampoon's Senior Trip (1995), with distance of 1.0:
12: Naked Gun 33 1/3: The Final Insult (1994), with distance of 1.0:


## Thank You !!